<a href="https://colab.research.google.com/github/SoniaSun810/LangChain_Practice/blob/main/Copy_of_LangChain_SQL_Database_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
! pip install langchain openai pymysql --upgrade -q

In [10]:
from secret_key import OpenAI_API_key
import os
os.environ["OPENAI_API_KEY"] = OpenAI_API_key

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

In [ ]:
db_user = "admin"
db_password = "MSWE-ta-project!"
db_host = "aws-ta-matching.cpxfig1ppqqw.eu-north-1.rds.amazonaws.com"
db_name = "TA_Matching_System"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0.5)

In [ ]:
toolkit = SQLDatabaseToolkit(db = db, llm = llm)

In [ ]:
agent_executor = create_sql_agent(
    llm = llm,
    toolkit = toolkit,
    verbose = True
)

In [16]:
agent_executor.run("Describe the Table Courses")



> Entering new  chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Courses, Students_basic_info, Students_candidacy
Thought:I can query the "Courses" table to describe its structure.
Action: sql_db_schema
Action Input: "Courses"
Observation: 
CREATE TABLE `Courses` (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	course_number VARCHAR(255), 
	course_name VARCHAR(255), 
	course_description TEXT, 
	prerequisites TEXT, 
	overlaps VARCHAR(255), 
	restrictions VARCHAR(255), 
	repeatability VARCHAR(255), 
	grading_options VARCHAR(255), 
	same_as VARCHAR(255), 
	roman_numerals VARCHAR(255), 
	PRIMARY KEY (id)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from Courses table:
id	course_number	course_name	course_description	prerequisites	overlaps	restrictions	repeatability	grading_options	same_as	roman_numerals
1	I&C SCI 9	Introduction to Computation for Scientists and Engineers	Introduces principles, techniques, and computational tools for quantitative ap

'The Courses table has the structure mentioned above and contains 3 sample rows.'

In [17]:
agent_executor.run("Which course would the student Smith1 be most suitable to be a Teaching Assistant for?")



> Entering new  chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Courses, Students_basic_info, Students_candidacy
Thought:I can query the Courses, Students_basic_info, and Students_candidacy tables to find the answer to the question.
Action: sql_db_schema
Action Input: "Courses, Students_basic_info, Students_candidacy"
Observation: 
CREATE TABLE `Courses` (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	course_number VARCHAR(255), 
	course_name VARCHAR(255), 
	course_description TEXT, 
	prerequisites TEXT, 
	overlaps VARCHAR(255), 
	restrictions VARCHAR(255), 
	repeatability VARCHAR(255), 
	grading_options VARCHAR(255), 
	same_as VARCHAR(255), 
	roman_numerals VARCHAR(255), 
	PRIMARY KEY (id)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from Courses table:
id	course_number	course_name	course_description	prerequisites	overlaps	restrictions	repeatability	grading_options	same_as	roman_numerals
1	I&C SCI 9	Introduction to Computation for Scientist

'CS 295P Keystone Project with instructor A+ in the quarter 202303.'

Tutorial, Langchain Beginner

---


1. promptTemplate

In [14]:
from langchain.prompts import PromptTemplate
prompt_template_name = PromptTemplate(
    input_variables = ['student_name'],
    template = "{student_name} is an applicant for Teaching Assistant"
)
prompt_template_name.format(student_name = "Smith1")

'Smith1 is an applicant for Teaching Assistant'

2. LLMChain

In [15]:
from langchain.chains import LLMChain

chain = LLMChain(llm = llm, prompt = prompt_template_name)
chain.run("Smith1")

"position at a university. He has a Bachelor's degree in Education and has completed his student teaching experience. Smith1 is passionate about teaching and has a strong desire to help students succeed academically. He has excellent communication skills and is able to effectively explain complex concepts to students in a way that they can understand. Smith1 is also highly organized and can manage multiple tasks efficiently. He is familiar with various teaching strategies and is able to adapt his teaching style to meet the needs of different students. Smith1 is dedicated to creating a positive and inclusive learning environment where all students feel valued and supported. He believes in the importance of building strong relationships with students and their families to foster a collaborative learning environment. Smith1 is committed to professional development and staying up-to-date with the latest research and trends in education. He is excited about the opportunity to contribute to 

3. Sequential Chain